In [ ]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils import to_categorical


In [ ]:
print(tf.keras.__version__)

In [ ]:
# Initialising the compression dimensions
img_width = 256
img_height =256
border = 5


In [ ]:
# Extracting the file names of the images and masks in training, test and validation folders


ids_train = next(os.walk("D:/ISIC2018_Task1-2_Training_Input_x2"))[2] # returns all the files "DIR."

ids_seg_train=next(os.walk("D:/ISIC2018_Task1_Training_GroundTruth_x2"))[2] # returns all the files "DIR."

print("No. of images in training folder= ",len(ids_train))
print("No. of images in test folder= ",len(ids_seg_train))


In [ ]:
ids_train_sort=sorted(ids_train) # Sorting of data wit respect to labels
ids_seg_train_sort=sorted(ids_seg_train) # Sorting of data wit respect to labels


In [ ]:
def load_image(inp_path,ids):
    X= np.zeros((len(ids),img_height,img_width,1),dtype=np.float32)
    for n, id_ in tqdm_notebook(enumerate(ids), total=len(ids)): # capture all the images ids using tqdm
       
        img = load_img(inp_path+id_, color_mode = 'grayscale')  # Load images here
        x_img = img_to_array(img) # Convert images to array
        x_img = resize(x_img,(256,256,1),mode = 'constant',preserve_range = True)
        X[n] = x_img/255 # Normalize the images
    return X

In [ ]:
def load_seg(inp_path,ids):
    X= np.zeros((len(ids),img_height,img_width,1),dtype=np.uint8)
    for n, id_ in tqdm_notebook(enumerate(ids), total=len(ids)):
        # Load images
        img = load_img(inp_path+id_,color_mode = 'grayscale')
        x_img = img_to_array(img)
        x_img = resize(x_img,(256,256,1),mode = 'constant', preserve_range = True)
        X[n] = x_img
    return X

In [ ]:
X_Oasis_train = load_image("D:/ISIC2018_Task1-2_Training_Input_x2/",ids_train_sort)

In [ ]:
Y_Oasis_train=load_seg("D:/ISIC2018_Task1_Training_GroundTruth_x2/",ids_seg_train_sort)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_Oasis_train, Y_Oasis_train, test_size=0.20, random_state=42)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [ ]:

Y_Oasis_train_sc = y_train//85
Y_Oasis_test_sc = y_test//85

In [ ]:
Y_Oasis_train_cat = to_categorical(Y_Oasis_train_sc) # one hot encoding
Y_Oasis_test_cat = to_categorical(Y_Oasis_test_sc) # one hot encoding
Y_Oasis_val_cat = to_categorical(Y_Oasis_val_sc) # one hot encoding